In [1]:
import os
import time
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
import Turbine
from scipy.optimize import root

RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")

water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    RP=RP,
)

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel(
    "streams0.xlsx", sheet_name="water", index_col=0)
# таблица задаваемых значений (заменится на интерфейс)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)

KPD_SP = 0.99
calctolerance = 10**-3
start_time = time.time()


class turboustanovka:
    def __init__(self, water, water_streams0, water_streams, KPD_SP=0.99):
        self.Tepl_systema = SP.teplofik_systema(
            KPD_SP, water,  water_streams0, water_streams)

        self.Turb = Turbine.turbine(water,
                                    "DROSVD-TURBVD",
                                    "ENDOFVD",
                                    "DROSND-TURBND",
                                    "SMESHEND",
                                    "DOOTB2",
                                    "DOOTB1",
                                    "INCND",
                                    "INKOND",
                                    water_streams0,
                                    water_streams,
                                    )
        self.water_streams = water_streams

    def calculate_t_rejim(self, calcmethod, calctolerance, maxiterations):
        
        
        
#         def Find_Potb2(diafragma):
#             Tepl_systema_res = self.Tepl_systema.calculate(
#                 maxiterations, calctolerance)
#             self.water_streams.at['DOOTB1',
#                                   'G'] =self.water_streams.at['DOOTB2',
#                                   'G']- max(0,self.water_streams.at['OTB2-SP2', 'G'])
            
#             self.water_streams.loc['INCND':'INKOND',
#                                   'G'] =self.water_streams.at['DOOTB1',
#                                   'G']- max(0,self.water_streams.at['OTB1-SP1', 'G'])
#             Turb_res = self.Turb.calculate(diafragma, maxiterations,
#                                            calctolerance)
            
#             self.water_streams.loc['OTB1-SP1', 'T':'H']=self.water_streams.loc['DOOTB1', 'T':'H']
#             self.water_streams.at['OTB2-SP2', 'T']=self.water_streams.at['DOOTB2', 'T']
#             self.water_streams.at['OTB2-SP2', 'H']=self.water_streams.at['DOOTB2', 'H']       
            
#             Potb2_turb = self.water_streams.at['DOOTB2', 'P']
#             Potb2_teplof = self.water_streams.at['OTB2-SP2', 'P']
#             Error_p = (Potb2_turb-Potb2_teplof)/Potb2_teplof*100
#             print('Diafragma',diafragma)
#             print(self.water_streams['SWIN-TURB':'INKOND'])

#             return Error_p
        
#         Solution_Potb2 = root(Find_Potb2, self.water_streams.at['OTB2-SP2', 'P'],
#                                       method=calcmethod, tol=calctolerance)
#         Diafragma = float(Solution_Potb2.x)

        Diafragma = 0
        for i in range(maxiterations):
            # for i in range(3):
            Tepl_systema_res = self.Tepl_systema.calculate(
                maxiterations, calctolerance)
            self.water_streams.at['DOOTB1',
                                  'G'] =max(1,self.water_streams.at['DOOTB2',
                                  'G']- self.water_streams.at['OTB2-SP2', 'G'])
                
            self.water_streams.loc['INCND':'INKOND',
                                  'G'] =max(1,self.water_streams.at['DOOTB1',
                                  'G']- self.water_streams.at['OTB1-SP1', 'G'])
            Turb_res = self.Turb.calculate(Diafragma, maxiterations,
                                           calctolerance)
            self.water_streams.loc['OTB1-SP1', 'T':'H']=self.water_streams.loc['DOOTB1', 'T':'H']
            self.water_streams.at['OTB2-SP2', 'T']=self.water_streams.at['DOOTB2', 'T']
            self.water_streams.at['OTB2-SP2', 'H']=self.water_streams.at['DOOTB2', 'H']   
            
            Potb2_turb = self.water_streams.at['DOOTB2', 'P']
            Potb2_teplof = self.water_streams.at['OTB2-SP2', 'P']
            Error_p = (Potb2_turb-Potb2_teplof)/Potb2_teplof*100
            Diafragma = max(0,Diafragma-Error_p/100)
            print('Diafragma',Diafragma)
            print(self.water_streams['SWIN-TURB':'INKOND'])

            if abs(Error_p) < calctolerance:
                print(
                    "Максимальная погрешность определения давления в верхнем отборе", Error_p)
                break
            if i == maxiterations-1:
                print(
                    'Достигнуто максимальное количество итераций давления верхнего отбора')

        Turb_res = self.Turb.calculate(Diafragma, maxiterations,
                                       calctolerance)
        Tepl_systema_res = self.Tepl_systema.calculate(
            maxiterations, calctolerance)

        Result = {'Turb_res': Turb_res,
                  'Tepl_systema_res': Tepl_systema_res, 'Diafragma': Diafragma}
        return Result

    def calculate(self, teplofikacia=0, calcmethod='hybr', calctolerance=10**-3, maxiterations=20,):

        if teplofikacia == 1:
            Result = self.calculate_t_rejim(
                calcmethod, calctolerance, maxiterations)

        if teplofikacia == 0:
            diafragma = 0
            Turb_res = self.Turb.calculate(diafragma, maxiterations,
                                           calctolerance)
            Result = {'Turb_res': Turb_res}
        return Result


start_time = time.time()
TU = turboustanovka(water, water_streams0, water_streams, KPD_SP)
TU_res = TU.calculate(teplofikacia=1, calcmethod='hybr',
                      calctolerance=10**-3, maxiterations=50)

print("fin:--- %s сек. ---" % round((time.time() - start_time), 1))

Мощность теплофикационной установки 112.27922030202504
Погрешность определения тепловой мощности теплофикационной установки -0.00026081158243219374
Максимальная погрешность определения давления в отборах 9.778595447468692e-05
Diafragma 0
                        T         P            H           G         S  \
SWIN-TURB       69.000000  1.000000          NaN  784.100000       NaN   
SWIN-OD         69.000000  1.000000          NaN   98.773793       NaN   
OD-SP1          88.727546  1.000000   371.502235   98.773793       NaN   
OTB2-SP2       139.416659  0.117338  2746.531269   -2.982756       NaN   
SP2-SP1        102.670149  0.111471   430.434324   -2.982756       NaN   
SP1-SP2        105.250207  1.000000   440.682619  784.100000       NaN   
SP2-WOUT       103.200000  1.000000   432.098400  784.100000       NaN   
WIN-SP1         71.485097  1.000000   192.602000  784.100000       NaN   
OTB1-SP1        51.623497  0.013382  2401.454293   50.230971       NaN   
DROSVD-TURBVD  492.143

C:\Users\Vladimir\Documents\GitHub\GZ\SP.py:35: RuntimeWarning: invalid value encountered in sqrt
  epsilon=min(0.999, 1/(0.35*min(Gdr,God)/max(Gdr,God)+0.65+1/self.F*n.sqrt(min(Gdr,God)/max(Gdr,God))))


Максимальная погрешность определения давления в отборах 0.0006021591749988777
Diafragma 0
                        T         P            H           G         S  \
SWIN-TURB       69.000000  1.000000          NaN  784.100000       NaN   
SWIN-OD         69.000000  1.000000          NaN   98.773793       NaN   
OD-SP1          75.570745  1.000000   316.414711   98.773793       NaN   
OTB2-SP2       139.365292  0.145799  2746.435851   32.363672       NaN   
SP2-SP1        108.974403  0.138509   457.075803   32.363672       NaN   
SP1-SP2         80.856523  1.000000   338.546263  784.100000       NaN   
SP2-WOUT       103.200000  1.000000   432.098400  784.100000       NaN   
WIN-SP1         69.827723  1.000000   192.602000  784.100000       NaN   
OTB1-SP1        48.072226  0.011218  2382.613386   16.085217       NaN   
DROSVD-TURBVD  492.143703  5.187858  3414.093682   41.970000  6.935022   
ENDOFVD        196.701623  0.400019  2854.001352   41.970000  7.157589   
DROSND-TURBND  200.886

C:\Users\Vladimir\Documents\GitHub\GZ\SP.py:35: RuntimeWarning: invalid value encountered in sqrt
  epsilon=min(0.999, 1/(0.35*min(Gdr,God)/max(Gdr,God)+0.65+1/self.F*n.sqrt(min(Gdr,God)/max(Gdr,God))))
C:\Users\Vladimir\Documents\GitHub\GZ\SP.py:35: RuntimeWarning: invalid value encountered in sqrt
  epsilon=min(0.999, 1/(0.35*min(Gdr,God)/max(Gdr,God)+0.65+1/self.F*n.sqrt(min(Gdr,God)/max(Gdr,God))))


Мощность теплофикационной установки 112.27918190605295
Погрешность определения тепловой мощности теплофикационной установки -0.00029500835707947393
Максимальная погрешность определения давления в отборах 0.0
Diafragma 0
                        T         P            H           G         S  \
SWIN-TURB       69.000000  1.000000          NaN  784.100000       NaN   
SWIN-OD         69.000000  1.000000          NaN   98.773793       NaN   
OD-SP1          58.973200  1.000000   246.920789   98.773793       NaN   
OTB2-SP2       139.860143  0.193427  2746.369967   81.746461       NaN   
SP2-SP1        117.552825  0.183756   493.421432   81.746461       NaN   
SP1-SP2         47.663172  1.000000   199.565702  784.100000       NaN   
SP2-WOUT       103.200000  1.000000   432.098400  784.100000       NaN   
WIN-SP1         67.736915  1.000000   192.602000  784.100000       NaN   
OTB1-SP1        91.466929  0.074184  2364.613485  -42.144122       NaN   
DROSVD-TURBVD  492.146319  5.188375  341

...

C:\Users\Vladimir\Documents\GitHub\GZ\SP.py:35: RuntimeWarning: invalid value encountered in sqrt
  epsilon=min(0.999, 1/(0.35*min(Gdr,God)/max(Gdr,God)+0.65+1/self.F*n.sqrt(min(Gdr,God)/max(Gdr,God))))
C:\Users\Vladimir\Documents\GitHub\GZ\SP.py:35: RuntimeWarning: invalid value encountered in sqrt
  epsilon=min(0.999, 1/(0.35*min(Gdr,God)/max(Gdr,God)+0.65+1/self.F*n.sqrt(min(Gdr,God)/max(Gdr,God))))


Максимальная погрешность определения давления в отборах 0.0
Diafragma 0
                        T         P            H           G         S  \
SWIN-TURB       69.000000  1.000000          NaN  784.100000       NaN   
SWIN-OD         69.000000  1.000000          NaN   98.773793       NaN   
OD-SP1          54.896784  1.000000   229.852835   98.773793       NaN   
OTB2-SP2       141.013315  0.228782  2746.219271  113.107241       NaN   
SP2-SP1        122.859040  0.217342   515.963977  113.107241       NaN   
SP1-SP2         27.131303  1.000000   113.598767  784.100000       NaN   
SP2-WOUT       103.200000  1.000000   432.098400  784.100000       NaN   
WIN-SP1         67.223405  1.000000   192.602000  784.100000       NaN   
OTB1-SP1       109.089211  0.139047  2229.188421  -84.796087       NaN   
DROSVD-TURBVD  492.153047  5.189707  3414.093682   41.970000  6.934864   
ENDOFVD        197.241622  0.423326  2853.931985   41.970000  7.131894   
DROSND-TURBND  201.441597  0.485326  285

C:\Users\Vladimir\Documents\GitHub\GZ\SP.py:35: RuntimeWarning: invalid value encountered in sqrt
  epsilon=min(0.999, 1/(0.35*min(Gdr,God)/max(Gdr,God)+0.65+1/self.F*n.sqrt(min(Gdr,God)/max(Gdr,God))))
C:\Users\Vladimir\Documents\GitHub\GZ\SP.py:35: RuntimeWarning: invalid value encountered in sqrt
  epsilon=min(0.999, 1/(0.35*min(Gdr,God)/max(Gdr,God)+0.65+1/self.F*n.sqrt(min(Gdr,God)/max(Gdr,God))))


Мощность теплофикационной установки 112.27933942383932
Погрешность определения тепловой мощности теплофикационной установки -0.00015471759346843782
Максимальная погрешность определения давления в отборах 0.0008595858598331606
Diafragma 0
                        T         P            H           G         S  \
SWIN-TURB       69.000000  1.000000          NaN  784.100000       NaN   
SWIN-OD         69.000000  1.000000          NaN   98.773793       NaN   
OD-SP1          89.633169  1.000000   375.294080   98.773793       NaN   
OTB2-SP2       139.249238  0.121213  2746.220338    2.209703       NaN   
SP2-SP1        103.598120  0.115152   434.352531    2.209703       NaN   
SP1-SP2        101.659515  1.000000   425.648388  784.100000       NaN   
SP2-WOUT       103.200000  1.000000   432.098400  784.100000       NaN   
WIN-SP1         71.599179  1.000000   192.602000  784.100000       NaN   
OTB1-SP1        20.064303  0.002349  2229.188356   56.086453       NaN   
DROSVD-TURBVD  492.143

C:\Users\Vladimir\Documents\GitHub\GZ\SP.py:35: RuntimeWarning: invalid value encountered in sqrt
  epsilon=min(0.999, 1/(0.35*min(Gdr,God)/max(Gdr,God)+0.65+1/self.F*n.sqrt(min(Gdr,God)/max(Gdr,God))))


In [2]:
water_streams['SWIN-TURB':'INKOND']

,T,P,H,G,S,X
SWIN-TURB,69.000000,1.000000,NaN,784.100000,NaN,NaN
SWIN-OD,69.000000,1.000000,NaN,98.773793,NaN,NaN
OD-SP1,89.633169,1.000000,375.294080,98.773793,NaN,NaN
OTB2-SP2,141.013315,0.121213,2746.219271,2.209703,NaN,NaN
SP2-SP1,103.598120,0.115152,434.352531,2.209703,NaN,NaN
SP1-SP2,101.659515,1.000000,425.648388,784.100000,NaN,NaN
SP2-WOUT,103.200000,1.000000,432.098400,784.100000,NaN,NaN
WIN-SP1,71.599179,1.000000,192.602000,784.100000,NaN,NaN
OTB1-SP1,109.089211,0.139047,2229.188421,56.086453,NaN,NaN
DROSVD-TURBVD,492.158519,5.190790,3414.093682,41.970000,6.934772,100.000000
